In [ ]:
# Installing JDK, Spark, findspark  and setting the environment variables

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark
 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"
 
!ls

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Reading package lists... Done
bank_prospects.csv  spark-3.3.1-bin-hadoop3	 spark-3.3.1-bin

In [ ]:
# Importing pyspark and creating the SparkSession object

import findspark
findspark.init()
 
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [ ]:
# Get the data

!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv

--2023-01-19 15:47:04--  https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv.1’

bank_prospects.csv. 100%[===================>]     306  --.-KB/s    in 0s      

2023-01-19 15:47:04 (9.26 MB/s) - ‘bank_prospects.csv.1’ saved [306/306]



In [ ]:
# See the content of the file

!cat bank_prospects.csv

Age,Salary,Gender,Country,Purchased
18,20000,Male,Germany,N
19,22000,Female,France,N
20,24000,Female,England,N
21,,Male,England,N
22,50000,Male,France,Y
23,35000,Female,England,N
24,,Male,Germany,N
25,32000,Female,France,Y
,35000,Male,Germany,N
27,37000,Female,France,N
27,37000,Female,unknown,N

In [ ]:
# Load the file to a dataframe

df = spark.read.csv("bank_prospects.csv", header=True)

In [ ]:
df.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



In [ ]:
# Filtering unknown values

df = df.filter(df['Country']!='unknown')

In [ ]:
df.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [ ]:
# Printing the schema of the dataframe

df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [ ]:
# Convert Age to integer type and Salary to float type

from pyspark.sql.types import IntegerType,FloatType
df2 = df.withColumn('Age',df['Age'].cast(IntegerType())).withColumn('Salary',df['Salary'].cast(FloatType()))

In [ ]:
# Calculating mean value of age and Salary

from pyspark.sql.functions import mean
mean_age_value = df2.select(mean('Age')).collect()
mean_salary_value = df2.select(mean('Salary')).collect()

mean_age = mean_age_value[0][0]
mean_salary = mean_salary_value[0][0]

In [ ]:
# Replacing null values in Age and Salary with their respective mean values.


df3 = df2.na.fill(mean_salary,'Salary')

In [ ]:
df3 = df3.na.fill(mean_age,'Age')

In [ ]:
df3.show()

+---+-------+------+-------+---------+
|Age| Salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [59]:
df3.write.option('header','true').format('csv').save('bank_propspects_transformed')

In [60]:
!ls

bank_propspects_transformed  spark-3.3.1-bin-hadoop3
bank_prospects.csv	     spark-3.3.1-bin-hadoop3.tgz
bank_prospects.csv.1	     spark-3.3.1-bin-hadoop3.tgz.1
sample_data		     temp


In [61]:
!ls bank_propspects_transformed

part-00000-2dc34e88-0696-4d68-8956-a3f652e64ed2-c000.csv  _SUCCESS


In [63]:
!cat bank_propspects_transformed/part-00000-2dc34e88-0696-4d68-8956-a3f652e64ed2-c000.csv

Age,Salary,Gender,Country,Purchased
18,20000.0,Male,Germany,N
19,22000.0,Female,France,N
20,24000.0,Female,England,N
21,31875.0,Male,England,N
22,50000.0,Male,France,Y
23,35000.0,Female,England,N
24,31875.0,Male,Germany,N
25,32000.0,Female,France,Y
22,35000.0,Male,Germany,N
27,37000.0,Female,France,N
